## 6.1: Sourcing Open Data

### This script contains the following:

#### 1. Importing  Libraries 
#### 2. Import Data
#### 3. check columns
#### 4. Finding and handling missing data
#### 5. checking datatypes
#### 6. Finding Duplicates
#### 7. Descriptive analysis
#### 8. Rename columns

<h2>1. Importing Libraries</h2>

In [47]:
# Import libraries
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime


## 2. Import Data

In [49]:

path = r'C:\Users\Lukman\Desktop\FEMINA CF\Data Immerssion\Achievement6\Analyzing Flight Delays'
df = pd.read_csv(os.path.join(path,'02 Data','Original Data','flightsdelay_data.csv'))

# Display the first few rows of the dataframe
print(df.head())

      FL_DATE                AIRLINE                AIRLINE_DOT AIRLINE_CODE  \
0  2019-01-09  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
1  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
2  2022-07-22  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
3  2023-03-06   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
4  2020-02-23       Spirit Air Lines       Spirit Air Lines: NK           NK   

   DOT_CODE  FL_NUMBER ORIGIN          ORIGIN_CITY DEST  \
0     19977       1562    FLL  Fort Lauderdale, FL  EWR   
1     19790       1149    MSP      Minneapolis, MN  SEA   
2     19977        459    DEN           Denver, CO  MSP   
3     19790       2295    MSP      Minneapolis, MN  SFO   
4     20416        407    MCO          Orlando, FL  DFW   

               DEST_CITY  ...  DIVERTED  CRS_ELAPSED_TIME  ELAPSED_TIME  \
0             Newark, NJ  ...       0.0             186.0         176.0   
1            S

## 3. Check for Column Names

In [50]:
print(df.columns)


Index(['FL_DATE', 'AIRLINE', 'AIRLINE_DOT', 'AIRLINE_CODE', 'DOT_CODE',
       'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'DELAY_DUE_CARRIER',
       'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY',
       'DELAY_DUE_LATE_AIRCRAFT'],
      dtype='object')


In [51]:
print(df.shape)

(3000000, 32)


## 4. Finding and Handling Missing Data

In [52]:
# Check for missing values
df.isnull().sum()


FL_DATE                          0
AIRLINE                          0
AIRLINE_DOT                      0
AIRLINE_CODE                     0
DOT_CODE                         0
FL_NUMBER                        0
ORIGIN                           0
ORIGIN_CITY                      0
DEST                             0
DEST_CITY                        0
CRS_DEP_TIME                     0
DEP_TIME                     77615
DEP_DELAY                    77644
TAXI_OUT                     78806
WHEELS_OFF                   78806
WHEELS_ON                    79944
TAXI_IN                      79944
CRS_ARR_TIME                     0
ARR_TIME                     79942
ARR_DELAY                    86198
CANCELLED                        0
CANCELLATION_CODE          2920860
DIVERTED                         0
CRS_ELAPSED_TIME                14
ELAPSED_TIME                 86198
AIR_TIME                     86198
DISTANCE                         0
DELAY_DUE_CARRIER          2466137
DELAY_DUE_WEATHER   

In [53]:
# Fill missing cancellation codes with a placeholder
df['CANCELLATION_CODE'].fillna('Not Canceled', inplace=True)

In [54]:
# Fill missing delay reasons with 0 (no delay)
delay_columns = [
    'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 
    'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT'
]
df[delay_columns] = df[delay_columns].fillna(0)

In [55]:
# drop rows with missing values if they are not canceled
df.dropna(subset=['DEP_TIME', 'ARR_TIME'], inplace=True)

In [56]:
# For `CRS_ELAPSED_TIME`, fill with the median value
df['CRS_ELAPSED_TIME'].fillna(df['CRS_ELAPSED_TIME'].median(), inplace=True)

In [57]:
#checking missing values 
df.isnull().sum()

FL_DATE                       0
AIRLINE                       0
AIRLINE_DOT                   0
AIRLINE_CODE                  0
DOT_CODE                      0
FL_NUMBER                     0
ORIGIN                        0
ORIGIN_CITY                   0
DEST                          0
DEST_CITY                     0
CRS_DEP_TIME                  0
DEP_TIME                      0
DEP_DELAY                     0
TAXI_OUT                      0
WHEELS_OFF                    0
WHEELS_ON                     2
TAXI_IN                       2
CRS_ARR_TIME                  0
ARR_TIME                      0
ARR_DELAY                  6256
CANCELLED                     0
CANCELLATION_CODE             0
DIVERTED                      0
CRS_ELAPSED_TIME              0
ELAPSED_TIME               6256
AIR_TIME                   6256
DISTANCE                      0
DELAY_DUE_CARRIER             0
DELAY_DUE_WEATHER             0
DELAY_DUE_NAS                 0
DELAY_DUE_SECURITY            0
DELAY_DU

## 5. Check Datatypes

In [58]:
# check for datatypes
df.dtypes

FL_DATE                     object
AIRLINE                     object
AIRLINE_DOT                 object
AIRLINE_CODE                object
DOT_CODE                     int64
FL_NUMBER                    int64
ORIGIN                      object
ORIGIN_CITY                 object
DEST                        object
DEST_CITY                   object
CRS_DEP_TIME                 int64
DEP_TIME                   float64
DEP_DELAY                  float64
TAXI_OUT                   float64
WHEELS_OFF                 float64
WHEELS_ON                  float64
TAXI_IN                    float64
CRS_ARR_TIME                 int64
ARR_TIME                   float64
ARR_DELAY                  float64
CANCELLED                  float64
CANCELLATION_CODE           object
DIVERTED                   float64
CRS_ELAPSED_TIME           float64
ELAPSED_TIME               float64
AIR_TIME                   float64
DISTANCE                   float64
DELAY_DUE_CARRIER          float64
DELAY_DUE_WEATHER   

In [59]:
# Function to check for mixed data types
def check_mixed_data_types(df):
    mixed_type_columns = []
    for col in df.columns:
        # Try to convert column to numeric, forcing non-numeric to NaN
        temp = pd.to_numeric(df[col], errors='coerce')
        # If the sum of NaNs is not zero and less than the length of the column, it indicates mixed types
        if temp.isnull().sum() > 0 and temp.isnull().sum() < len(temp):
            mixed_type_columns.append(col)
    return mixed_type_columns

# Identify columns with mixed data types
mixed_type_columns = check_mixed_data_types(df)
print(f"Columns with mixed data types: {mixed_type_columns}")

Columns with mixed data types: ['WHEELS_ON', 'TAXI_IN', 'ARR_DELAY', 'ELAPSED_TIME', 'AIR_TIME']


In [60]:
#Handling Mixed Data type
# Convert columns to numeric, coercing errors to NaN
for col in mixed_type_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Inspect the number of NaNs introduced by coercion
print(f"NaNs introduced after conversion:\n{df[mixed_type_columns].isna().sum()}")

# Handle NaNs: Fill with median value for each column
for col in mixed_type_columns:
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)

# Verify the changes
print(f"Columns with mixed data types after handling:\n{check_mixed_data_types(df)}")


NaNs introduced after conversion:
WHEELS_ON          2
TAXI_IN            2
ARR_DELAY       6256
ELAPSED_TIME    6256
AIR_TIME        6256
dtype: int64
Columns with mixed data types after handling:
[]


In [61]:
#checking missing values 
df.isnull().sum()

FL_DATE                    0
AIRLINE                    0
AIRLINE_DOT                0
AIRLINE_CODE               0
DOT_CODE                   0
FL_NUMBER                  0
ORIGIN                     0
ORIGIN_CITY                0
DEST                       0
DEST_CITY                  0
CRS_DEP_TIME               0
DEP_TIME                   0
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON                  0
TAXI_IN                    0
CRS_ARR_TIME               0
ARR_TIME                   0
ARR_DELAY                  0
CANCELLED                  0
CANCELLATION_CODE          0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE                   0
DELAY_DUE_CARRIER          0
DELAY_DUE_WEATHER          0
DELAY_DUE_NAS              0
DELAY_DUE_SECURITY         0
DELAY_DUE_LATE_AIRCRAFT    0
dtype: int64

## 6. Finding Duplicates

In [62]:
#Check for duplicate records
df.duplicated().sum()

0

In [63]:
# Columns to potentially drop
columns_to_drop = [
     'DOT_CODE',
    'WHEELS_OFF', 'WHEELS_ON', 'TAXI_OUT', 
    'TAXI_IN'
]

# Drop columns
df_cleaned = df.drop(columns=columns_to_drop)

## 7. Descriptive Analysis

In [64]:
df_cleaned.describe()

,FL_NUMBER,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
count,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2920058.0,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06,2.920058e+06
mean,2.508223e+03,1.326088e+03,1.329660e+03,1.008084e+01,1.489969e+03,1.466511e+03,4.236733e+00,0.0,2.142423e-03,1.424570e+02,1.365849e+02,1.122738e+02,8.110835e+02,4.526609e+00,7.286098e-01,2.406857e+00,2.667995e-02,4.656817e+00
std,1.746049e+03,4.854828e+02,4.992750e+02,4.912334e+01,5.109341e+02,5.318383e+02,5.112263e+01,0.0,4.623672e-02,7.174253e+01,7.160312e+01,6.968468e+01,5.895133e+02,3.214587e+01,1.394360e+01,1.506449e+01,1.532658e+00,2.579744e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,-9.000000e+01,1.000000e+00,1.000000e+00,-9.600000e+01,0.0,0.000000e+00,1.000000e+00,1.500000e+01,8.000000e+00,2.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.049000e+03,9.150000e+02,9.160000e+02,-6.000000e+00,1.107000e+03,1.053000e+03,-1.600000e+01,0.0,0.000000e+00,9.000000e+01,8.400000e+01,6.100000e+01,3.790000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.149000e+03,1.318000e+03,1.323000e+03,-2.000000e+00,1.515000e+03,1.505000e+03,-7.000000e+00,0.0,0.000000e+00,1.250000e+02,1.200000e+02,9.500000e+01,6.550000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.791000e+03,1.730000e+03,1.739000e+03,6.000000e+00,1.918000e+03,1.913000e+03,7.000000e+00,0.0,0.000000e+00,1.730000e+02,1.670000e+02,1.420000e+02,1.047000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.562000e+03,2.359000e+03,2.400000e+03,2.966000e+03,2.400000e+03,2.400000e+03,2.934000e+03,0.0,1.000000e+00,7.050000e+02,7.390000e+02,6.920000e+02,5.812000e+03,2.934000e+03,1.653000e+03,1.741000e+03,1.185000e+03,2.557000e+03


In [65]:
df_cleaned.shape

(2920058, 27)

## 8. Rename columns

In [66]:

df_cleaned.rename(columns={
    'FL_DATE': 'Flight_Date',
    'AIRLINE': 'Airline',
    'AIRLINE_DOT': 'Airline_DOT_Code',
    'AIRLINE_CODE': 'Airline_Code',
    'DOT_CODE': 'DOT_Code',
    'FL_NUMBER': 'Flight_Number',
    'ORIGIN': 'Origin_Airport',
    'ORIGIN_CITY': 'Origin_City',
    'DEST': 'Destination_Airport',
    'DEST_CITY': 'Destination_City',
    'CRS_DEP_TIME': 'Scheduled_Departure_Time',
    'DEP_TIME': 'Actual_Departure_Time',
    'DEP_DELAY': 'Departure_Delay',
    'TAXI_OUT': 'Taxi_Out_Time',
    'WHEELS_OFF': 'Wheels_Off_Time',
    'WHEELS_ON': 'Wheels_On_Time',
    'TAXI_IN': 'Taxi_In_Time',
    'CRS_ARR_TIME': 'Scheduled_Arrival_Time',
    'ARR_TIME': 'Actual_Arrival_Time',
    'ARR_DELAY': 'Arrival_Delay',
    'CANCELLED': 'Cancelled',
    'CANCELLATION_CODE': 'Cancellation_Code',
    'DIVERTED': 'Diverted',
    'CRS_ELAPSED_TIME': 'Scheduled_Elapsed_Time',
    'ELAPSED_TIME': 'Actual_Elapsed_Time',
    'AIR_TIME': 'Air_Time',
    'DISTANCE': 'Distance',
    'DELAY_DUE_CARRIER': 'Carrier_Delay',
    'DELAY_DUE_WEATHER': 'Weather_Delay',
    'DELAY_DUE_NAS': 'NAS_Delay',
    'DELAY_DUE_SECURITY': 'Security_Delay',
    'DELAY_DUE_LATE_AIRCRAFT': 'Late_Aircraft_Delay'
}, inplace=True)


In [67]:
df_cleaned.columns

Index(['Flight_Date', 'Airline', 'Airline_DOT_Code', 'Airline_Code',
       'Flight_Number', 'Origin_Airport', 'Origin_City', 'Destination_Airport',
       'Destination_City', 'Scheduled_Departure_Time', 'Actual_Departure_Time',
       'Departure_Delay', 'Scheduled_Arrival_Time', 'Actual_Arrival_Time',
       'Arrival_Delay', 'Cancelled', 'Cancellation_Code', 'Diverted',
       'Scheduled_Elapsed_Time', 'Actual_Elapsed_Time', 'Air_Time', 'Distance',
       'Carrier_Delay', 'Weather_Delay', 'NAS_Delay', 'Security_Delay',
       'Late_Aircraft_Delay'],
      dtype='object')

In [68]:
df_cleaned.to_csv(os.path.join(path, '02 Data','Prepared Data', 'flightdelay_cleaned.csv'))